# Table
Construction of m*n game board saved in tab numpy aray .

In [52]:
import numpy as np
import pandas as pd
m=5
n=5
tab=np.zeros([m,n],dtype=int)
obs=np.zeros([m,n],dtype=int)
obs[1:,0]=1
obs[0,3:n-2]=1
obs[2:3,3]=1
obs[0,0:5]=1
obs[2,4]=1
obs[4,1:3]=1
tab[obs==1]=-1

In [53]:
hardness='easy'

In [54]:
df=pd.read_csv(hardness+'/tab.csv')
tab=np.array(df.values.tolist())
df=pd.read_csv(hardness+'/constrain.csv')
constrain=df.values.tolist()


Table display

In [55]:
tab

array([[-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1,  0,  0,  0, -1,  0,  0],
       [-1,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  0, -1,  0,  0, -1, -1],
       [-1, -1,  0,  0, -1,  0,  0, -1],
       [-1, -1, -1,  0,  0, -1,  0,  0],
       [-1,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  0, -1,  0,  0,  0, -1]])

constrain saved in a list named as constrain .

In [47]:
constrain=[]
c=[1,0,2,False,6] #[[i,j],n,(True is vertical ) or (False is herozintal),sum]
constrain.append(c)
c=[2,3,2,True,5]
constrain.append(c)

checking if table hoped to be solved ?

In [30]:
def constrain_hope(table,constrain):
    if constrain==True:
        return True
    (i,j)=(constrain[0],constrain[1])
    n=constrain[2]
    sum=constrain[4]
    if constrain[3]:
        r=table[i+1:i+1+n,j]      
    else:
        r=table[i,j+1:j+1+n]
    if (np.unique(r[r!=0]).size)<(r[r!=0].size):
        return False
    if sum<np.sum(r):
        return False
    return True

checking if is completely solved ?

In [15]:
def tab_solved(table,constrain):
   for c in constrain:
      (i,j)=(c[0],c[1])
      n=c[2]
      sum=c[4]
      if c[3]:
         r=table[i+1:i+1+n,j]      
      else:
         r=table[i,j+1:j+1+n]
      if not(sum==np.sum(r)):
         return False
      return True

searching a cell constrain in raow and column 

In [16]:
def search_constrain_v(constrain,x,y):
    for i in constrain:
        if i[3]==True and i[1]==y and x>i[0] and x<=i[0]+i[2]:
            return i
    return True
def search_constrain_h(constrain,x,y):
    for i in constrain:
        if i[3]==False and i[0]==x and y>i[1] and y<=i[1]+i[2]:
            return i
    return True

# search
The below function is to search in state space .

In [17]:
def search(tab,constrain,selection):
    r,c=selection(tab)
    if r==-1:
        if tab_solved(tab,constrain):
            return True
        else :
            return False
    c1=search_constrain_v(constrain,r,c)
    c2=search_constrain_h(constrain,r,c)
    for i in range(1,5):
        tab[r,c]=i
        if not(constrain_hope(tab,c1) and constrain_hope(tab,c2)):
            tab[r,c]=0
            continue
        if search(tab,constrain,selection):
            return True
        tab[r,c]=0
    return False

# primary agent
selection in order :

In [18]:
def select_cell(tab):
    if not 0 in tab:
        return -1,-1
    else:
        for i in range(tab.shape[0]):
            for j in range(tab.shape[1]):
                if tab[i,j]==0:
                    return i,j

In [56]:
tab1=tab.copy()
search(tab1,constrain,selection=select_cell)

False

In [57]:
tab1

array([[-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1,  0,  0,  0, -1,  0,  0],
       [-1,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  0, -1,  0,  0, -1, -1],
       [-1, -1,  0,  0, -1,  0,  0, -1],
       [-1, -1, -1,  0,  0, -1,  0,  0],
       [-1,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  0, -1,  0,  0,  0, -1]])

selction in random

In [8]:
def select_cell_random(tab):
    if not 0 in tab:
        return -1,-1
    else:
        while True:
            a=np.random.randint(0,tab.shape[0],size=1)
            b=np.random.randint(0,tab.shape[1],size=1)
            if tab[a,b]==0:
                return a,b

In [9]:
tab2=tab.copy()
search(tab2,constrain,selection=select_cell_random)

True

In [10]:
tab2

array([[-1, -1, -1, -1, -1],
       [-1,  4,  2,  1,  1],
       [-1,  1,  1, -1, -1],
       [-1,  1,  1,  2,  1],
       [-1, -1, -1,  1,  1]])

# Intelligent agent

In [48]:
def select_cell_intelligent(tab,constrain=constrain):
    m=select_cell(tab)
    cv_m=search_constrain_v(constrain,m[0],m[1])
    ch_m=search_constrain_h(constrain,m[0],m[1])
    if not 0 in tab:
        return -1,-1
    else:
        for i in range(tab.shape[0]):
            for j in range(tab.shape[1]):
                if tab[i,j]==0:
                    cv=search_constrain_v(constrain,i,j)
                    ch=search_constrain_h(constrain,i,j)
                    if ((not type(cv)==bool) and (not type(cv_m)==bool) and cv[4]<cv_m[4]) or ((not type(ch)==bool) and (not type(ch_m)==bool) and ch[4]<ch_m[4]):
                        m=(i,j)
                        cv_m=cv
                        ch_m=ch
                    
    return m
                    

In [50]:
tab3=tab.copy()
search(tab3,constrain,selection=select_cell_intelligent)

True

In [51]:
tab3

array([[-1, -1, -1, -1, -1],
       [-1,  2,  4,  1,  1],
       [-1,  1,  1, -1, -1],
       [-1,  1,  1,  1,  1],
       [-1, -1, -1,  2,  1]])